In [79]:
import scanpy as sc
import os
import anndata as ad
import cellrank as cr
import numpy as np
import scipy as sp

In [194]:
# DATA_PATH="../../data/simulated/Synthetic_1000/dyn-SW/dyn-SW-1000-2/"
DATA_PATH="../../data/simulated/Synthetic_1000/dyn-LI/dyn-LI-1000-2/"
# DATA_PATH="../../data/simulated/Synthetic_1000/dyn-CY/dyn-CY-1000-2/"

In [195]:
adata = ad.read_h5ad(os.path.join(DATA_PATH, "anndata.h5ad"))

In [196]:
k = adata.uns['neighbors']['params']['n_neighbors']

In [197]:
del adata.uns['neighbors']

In [198]:
# dpt is true time, dpt_pseudotime is diffusion pseudotime
adata.obsm["X_dpt"] = adata.obs["dpt"].to_numpy().reshape(-1, 1)

In [199]:
sc.pp.neighbors(adata, k, use_rep = "X_dpt")

In [200]:
def compute_velocity_kernel(k, sigma):
    # compute velocity-induced transition matrix with kernel k and bandwidth sigma
    if sigma is None:
        # median heuristic, see e.g. https://github.com/theislab/cellrank/blob/f6d281e62a5cee976d25973b5a5e4c1050f37429/cellrank/kernels/_velocity_kernel.py#L230 
        logits_all = np.concatenate([k(adata.obsm["v"].iloc[i, :].to_numpy().reshape(1, -1).astype(np.float64),
                        (adata.X[adata.uns['neighbors']['connectivities'][i, :].indices, :] - adata.X[i, :]).astype(np.float64),
                                softmax_scale =  1.0)[1] for i in range(adata.shape[0])])
        sigma = 1/np.median(logits_all)
    probs_all_fwd = [k(adata.obsm["v"].iloc[i, :].to_numpy().reshape(1, -1).astype(np.float64),
                    (adata.X[adata.uns['neighbors']['connectivities'][i, :].indices, :] - adata.X[i, :]).astype(np.float64),
                    softmax_scale = sigma)[0] for i in range(adata.shape[0])]
    P_velo = sp.sparse.csr_matrix(adata.uns['neighbors']['connectivities'].shape)
    for i, p in enumerate(probs_all_fwd):
        P_velo[i, adata.uns['neighbors']['connectivities'][i, :].indices] = p
    return P_velo

# apply velocity kernels 
for (k, sigma, kerneltype) in zip([cr.tl.kernels.DotProductScheme(), cr.tl.kernels.CorrelationScheme(), cr.tl.kernels.CosineScheme()],
                            [None, None, None], ["dot", "corr", "cos"]):
    adata.obsm["P_velo_%s" % kerneltype] = compute_velocity_kernel(k, sigma)
# diffusion pseudotime kernel
k = cr.tl.kernels.PseudotimeKernel(adata, time_key = "dpt_pseudotime").compute_transition_matrix(threshold_scheme = "soft") 
adata.obsm["P_dpt"] = k.transition_matrix

  0%|          | 0/1000 [00:00<?, ?cell/s]

In [201]:
adata.obsm["P_velo_dot"][0, :]

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [202]:
# save transition matrices
for k in [x for x in adata.obsm.keys() if "P_" in x]:
    print("Writing transition matrix %s..." % k)
    try:
        np.save(os.path.join(DATA_PATH, f"{k}_dpt_knn.npy"), np.array(adata.obsm[k].todense()))
    except:
        np.save(os.path.join(DATA_PATH, f"{k}_dpt_knn.npy"), np.array(adata.obsm[k]))

Writing transition matrix P_dpt...
Writing transition matrix P_statot...
Writing transition matrix P_statot_ent...
Writing transition matrix P_velo_corr...
Writing transition matrix P_velo_cos...
Writing transition matrix P_velo_dot...
